In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import edward as ed
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

from os import listdir
from os.path import isfile, join
import os.path
import random

from edward.models import Poisson,Gamma

plt.style.use('ggplot')

In [2]:
def movielens(binary,threshold):
    file_path = 'data/movielens/X.txt'
    train_mask_file = 'data/movielens/train_mask.txt'
    test_mask_file = 'data/movielens/test_mask.txt'
    X = np.loadtxt(file_path)
    x_train = np.array(X)
    train_mask = np.loadtxt(train_mask_file)
    test_mask = np.loadtxt(test_mask_file)
    
    for i in range(0,X.shape[0]):
        for j in range(0,X.shape[1]):
            if train_mask[i][j] == 0:
                x_train[i][j] = 0

    
    if binary == False:
        return x_train,X,test_mask
    
    for i in range(0,X.shape[0]):
        for j in range(0,X.shape[1]):
            if x_train[i][j] <= threshold:
                x_train[i][j] = 0
            else:
                x_train[i][j] = 1
                
    return x_train,X,test_mask

In [3]:
x_train,X_full,test_mask = movielens(True,0)
k = 20

In [8]:
users = X_full.shape[0]
movies = X_full.shape[1]
a = a_c = c = c_c = 0.3
b_c = d_c = 1.0
scale = 0.0001

eu = Gamma(a_c*tf.ones([users,k]),(a_c/b_c)*tf.ones([users,k]))
ni = Gamma(c_c*tf.ones([movies,k]),(c_c/d_c)*tf.ones([movies,k]))
theta = Gamma(a*tf.ones([users,k]),eu)
beta = Gamma(c*tf.ones([movies,k]),ni)
lam1 = tf.matmul(theta,beta,transpose_b=True)
x = Poisson(lam = lam1,value = tf.ones([users,movies]))

q_eu = Gamma(tf.nn.softplus(tf.Variable(a_c*tf.ones([users,k]))),tf.nn.softplus(tf.Variable((a_c/b_c)*tf.ones([users,k]))))
q_ni = Gamma(tf.nn.softplus(tf.Variable(c_c*tf.ones([movies,k]))),tf.nn.softplus(tf.Variable((c_c/d_c)*tf.ones([movies,k]))))
q_theta = Gamma(tf.nn.softplus(tf.Variable(a*tf.ones([users,k]))),tf.nn.softplus(tf.Variable(eu)))
q_beta = Gamma(tf.nn.softplus(tf.Variable(c*tf.ones([movies,k]))),tf.nn.softplus(tf.Variable(ni)))


#q_eu = Gamma(tf.Variable(a_c*tf.ones([users,k])),tf.Variable((a_c/b_c)*tf.ones([users,k])))
#q_ni = Gamma(tf.Variable(c_c*tf.ones([movies,k])),tf.Variable((c_c/d_c)*tf.ones([movies,k])))
#q_theta = Gamma(tf.Variable(a*tf.ones([users,k])),tf.Variable(eu))
#q_beta = Gamma(tf.Variable(c*tf.ones([movies,k])),tf.Variable(ni))

inference = ed.KLqp({eu:q_eu,ni:q_ni,theta: q_theta,beta: q_beta},data={x:x_train})
inference.initialize(n_samples=1,scale={eu:scale,ni:scale,theta:scale,beta:scale})


init = tf.global_variables_initializer()
init.run()
#inference.run(n_iter = 300,n_print=1)
for i in range (1,inference.n_iter):
    inference.update()
    inference.9
    #if i%10 == 0:
     #   print(i)


300/300 [100%] ██████████████████████████████ Elapsed: 13s | Loss: nan


In [5]:
print(q_theta.sample(1).eval())

[[[ nan  nan  nan ...,  nan  nan  nan]
  [ nan  nan  nan ...,  nan  nan  nan]
  [ nan  nan  nan ...,  nan  nan  nan]
  ..., 
  [ nan  nan  nan ...,  nan  nan  nan]
  [ nan  nan  nan ...,  nan  nan  nan]
  [ nan  nan  nan ...,  nan  nan  nan]]]
